In [45]:
import numpy as np
from mindquantum.core.gates import X, Y, Z, S, H, RX, RY, RZ, Measure, UnivMathGate, DepolarizingChannel
from mindquantum.core.circuit import Circuit
from mindquantum.simulator import Simulator
import scipy.linalg as la
import matplotlib.pyplot as plt
from tqdm import tqdm
import concurrent.futures
import os
from scipy.optimize import curve_fit   

In [ ]:
def SpinChain(n,J,h):
    X = np.array([[0, 1], [1, 0]],dtype=complex)
    Y = np.array([[0, -1j], [1j, 0]],dtype=complex)
    Z = np.array([[1, 0], [0, -1]],dtype=complex)
    def hopping(P,i):
        assert i < n, "i should be less than n"
        if i == 0 or i == n-1:
            matrix = P
        else:
            matrix = np.eye(2,dtype=complex)
        for j in range(1,n):
            if j == i or j == i+1:
                matrix = np.kron(P,matrix)
            else:
                matrix = np.kron(np.eye(2,dtype=complex),matrix)
        return matrix
    def potential(P,i):
        assert i < n, "i should be less than n"
        if i == 0:
            matrix = P
        else:
            matrix = np.eye(2,dtype=complex)
        for j in range(1,n):
            if j == i:
                matrix = np.kron(P,matrix)
            else:
                matrix = np.kron(np.eye(2,dtype=complex),matrix)
        return matrix
    
    # hopping term
    HoppingX = np.zeros((2**n,2**n),dtype=complex)
    HoppingY = np.zeros((2**n,2**n),dtype=complex)
    HoppingZ = np.zeros((2**n,2**n),dtype=complex)
    if n == 2:
        for i in range(n-1):
            HoppingX += hopping(X,i)*J[0]
            HoppingY += hopping(Y,i)*J[1]
            HoppingZ += hopping(Z,i)*J[2]
    else:
        for i in range(n):
            HoppingX += hopping(X,i)*J[0]
            HoppingY += hopping(Y,i)*J[1]
            HoppingZ += hopping(Z,i)*J[2]
    # potential term
    PotentialX = np.zeros((2**n,2**n),dtype=complex)
    PotentialY = np.zeros((2**n,2**n),dtype=complex)
    PotentialZ = np.zeros((2**n,2**n),dtype=complex)
    for i in range(n):
        PotentialX += potential(X,i)*h[0]
        PotentialY += potential(Y,i)*h[1]
        PotentialZ += potential(Z,i)*h[2]
    return HoppingX+HoppingY+HoppingZ+PotentialX+PotentialY+PotentialZ

def TimeEvolution(H,t):
    eigenv,U = np.linalg.eig(H)
    diag = np.diag(np.exp(-1.j*t*eigenv))
    return U@diag@np.linalg.inv(U)

def gaussian(x, amplitude, mean, stddev):
    return amplitude * np.exp(-((x - mean) ** 2) / (2 * stddev ** 2))

def Diag_averg(matrix_set):
    L = matrix_set.shape[0]
    K = matrix_set.shape[1]
    N = L+K-1
    Y = matrix_set
    def sum(seq):
        a = seq[0]
        for i in range(1,len(seq)):
            a += seq[i]
        return a     
    recovered = []
    for i in range(N):
        if i<=L-1:
            seq = [Y[m,i-m]/(i+1) for m in range(i+1)]
            recovered.append(sum(seq))
        elif i>L-1 and i<K-1:
            seq = [Y[m,i-m]/L for m in range(L)]
            recovered.append(sum(seq))
        else:
            seq = [Y[m,i-m]/(N-i) for m in range(i-K+1,L)]
            recovered.append(sum(seq))
    return np.array(recovered)

def TimeSeqRecover(time_sequence_error,order):
    N = len(time_sequence_error)
    L = N//2
    K = N-L+1
    X = []
    for i in range(K):
        X.append(time_sequence_error[i:i+L])
    X = np.array(X).T
    U,Lambda,V = np.linalg.svd(X)
    r = order
    Y = U[:,:r]@np.diag(Lambda[:r])@V[:r,:]
    # print(Lambda[:2*r])
    time_sequence_recover = Diag_averg(Y)
    return time_sequence_recover

def QSA(Hamiltonian,ground_state,error=0.0,size=1000,with_SSA=True):
    n = int(np.log2(Hamiltonian.shape[0]))
    target_bit = [i+1 for i in range(n)]
    sim = Simulator('mqvector',n+1)

    sigma = 4.0
    points = 40
    width = 2*sigma

    time_stamp = np.linspace(-width,width,points+1)
    coeffs = [(width*2/points)*np.exp(-time_stamp[j]**2/(2*sigma**2))/(sigma*np.sqrt(2*np.pi)) for j in range(points+1)]

    U0 = TimeEvolution(Hamiltonian,-width)
    dU = TimeEvolution(Hamiltonian,2*width/points)
    Uset = []
    for i in tqdm(range(points+1)):
        randmat = np.random.randn(2**n,2**n)+1.j*np.random.randn(2**n,2**n)
        randmat = randmat*error
        Uset.append(U0)
        U0 = (dU+randmat)@U0
    Uset = np.array(Uset)

    # time_serial
    time_serial = []
    for i in tqdm(range(points+1),desc='I',colour='yellow'): # t
        realcirc = Circuit()
        realcirc += H.on(0)
        evol_gate = UnivMathGate('Ut',Uset[i])
        realcirc += evol_gate.on(target_bit,0)
        realcirc += H.on(0)
        realcirc += Measure('q0').on(0)
        
        sim.reset()
        sim.set_qs(np.kron(ground_state,[1.0,0.0]))
        result = sim.sampling(circuit=realcirc,shots=size)
        samples = result.data
        try:
            zero = samples['0']
        except:
            zero = 0
        try:
            one = samples['1']
        except:
            one = 0
        real_part = (zero-one)/size

        imagcirc = Circuit()
        imagcirc += H.on(0)
        evol_gate = UnivMathGate('Ut',Uset[i])
        imagcirc += evol_gate.on(target_bit,0)
        imagcirc += S.on(0)
        imagcirc += Z.on(0)
        imagcirc += H.on(0)
        imagcirc += Measure('q0').on(0)

        sim.reset()
        sim.set_qs(np.kron(ground_state,[1.0,0.0]))
        result = sim.sampling(circuit=imagcirc,shots=size)
        samples = result.data
        try:
            zero = samples['0']
        except:
            zero = 0
        try:
            one = samples['1']
        except:
            one = 0
        imag_part = (zero-one)/size

        time_serial.append(real_part+1.j*imag_part)

        del realcirc
        del imagcirc  
    time_serial = np.array(time_serial) 

    # post - processing
    if with_SSA:
        time_serial_recover = TimeSeqRecover(time_serial,2)
    else:
        time_serial_recover = time_serial
    energy_stamp = np.linspace(-23,-2,4319)

    amplitude = []
    for energy in energy_stamp:
        coeffs = []
        for i in range(points+1):
            coeffs.append((width*2/points)*np.exp(-time_stamp[i]**2/(2*sigma**2)+1.j*energy*time_stamp[i])/(sigma*np.sqrt(2*np.pi)))
        amplitude.append(np.array(coeffs)@time_serial_recover)
    try:
        # plt.plot(energy_stamp[:600],np.real(amplitude)[:600])
        popt, pcov = curve_fit(gaussian, energy_stamp[:600], np.real(amplitude[:600]), p0=[0.1,-21.5,1])
        estimated_energy = popt[1]
    except:
        pass

    return estimated_energy

In [124]:
n = 8
size = 1000
J = [-1.0,-1.0,-1.5]
h = [1.5,0.0,0.5]
Hamiltonian = SpinChain(n,J,h)
ground_state = np.linalg.eig(Hamiltonian)[1][:,0]
ground_energy = np.linalg.eigvalsh(Hamiltonian)[0]

In [125]:
# with SSA
dU_error = [1e-1,1e-2,1e-3,1e-4,1e-5,0]
samples = []
for i in tqdm(range(10),desc='samples',colour='blue'):
    energy_error = []
    for e in tqdm(dU_error,desc='error',colour='red'):
        estimate_energy = QSA(Hamiltonian=Hamiltonian,ground_state=ground_state,error=e,with_SSA=True)
        energy_error.append(estimate_energy-ground_energy)
    samples.append(energy_error)

samples:   0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 41/41 [00:00<00:00, 222.66it/s]






I: 100%|██████████| 41/41 [00:00<00:00, 63.11it/s]


100%|██████████| 41/41 [00:00<00:00, 229.97it/s]






I: 100%|██████████| 41/41 [00:00<00:00, 66.10it/s]


100%|██████████| 41/41 [00:00<00:00, 249.77it/s]






I: 100%|██████████| 41/41 [00:00<00:00, 68.88it/s]


100%|██████████| 41/41 [00:00<00:00, 250.38it/s]






I: 100%|██████████| 41/41 [00:00<00:00, 65.11it/s]


100%|██████████| 41/41 [00:00<00:00, 223.38it/s]






I: 100%|██████████| 41/41 [00:00<00:00, 64.52it/s]


100%|██████████| 41/41 [00:00<00:00, 222.85it/s]






samples:  10%|█         | 1/10 [00:09<01:23,  9.25s/it]

100%|██████████| 41/41 [00:00<00:00, 216.71it/s]






I: 100%|██████████| 41/41 [00:00<00:00, 62.55it/s]


100%|██████████| 41/41 [00:00<00:00, 244.96it/s]






I: 100%|██████████| 41/41 [00:00<00:00, 67.94it/s]


100%|██████████| 41/41 [00:00<00:00, 238.37it/s]






I: 100%|██████████| 41/41 [0

In [126]:
post_samples = np.array(np.abs(samples)).T
benchmark = []
for i in range(len(dU_error)):
    benchmark.append((np.mean(post_samples[i]),np.std(post_samples[i])))
benchmark = np.array(benchmark)
print('mean:',benchmark[:,0])
print('std:',benchmark[:,1])

mean: [1.42648496e+00 5.05650603e-03 9.68094887e-04 6.45132002e-04
 1.12625995e-03 9.35553723e-04]
std: [0.00318994 0.00251954 0.00063573 0.00037576 0.00093077 0.00086506]


In [127]:
# without SSA
dU_error = [1e-1,1e-2,1e-3,1e-4,1e-5,0]
samples = []
for i in tqdm(range(10),desc='samples',colour='blue'):
    energy_error = []
    for e in tqdm(dU_error,desc='error',colour='red'):
        estimate_energy = QSA(Hamiltonian=Hamiltonian,ground_state=ground_state,error=e,with_SSA=False)
        energy_error.append(estimate_energy-ground_energy)
    samples.append(energy_error)

samples:   0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 41/41 [00:00<00:00, 231.16it/s]






I: 100%|██████████| 41/41 [00:00<00:00, 66.00it/s]


100%|██████████| 41/41 [00:00<00:00, 212.44it/s]






I: 100%|██████████| 41/41 [00:00<00:00, 62.60it/s]



100%|██████████| 41/41 [00:00<00:00, 190.21it/s]






I: 100%|██████████| 41/41 [00:00<00:00, 63.52it/s]


100%|██████████| 41/41 [00:00<00:00, 197.12it/s]







I: 100%|██████████| 41/41 [00:00<00:00, 63.32it/s]


100%|██████████| 41/41 [00:00<00:00, 216.93it/s]






I: 100%|██████████| 41/41 [00:00<00:00, 62.10it/s]


100%|██████████| 41/41 [00:00<00:00, 230.36it/s]






samples:  10%|█         | 1/10 [00:09<01:26,  9.57s/it]


100%|██████████| 41/41 [00:00<00:00, 201.37it/s]






I: 100%|██████████| 41/41 [00:00<00:00, 67.55it/s]


100%|██████████| 41/41 [00:00<00:00, 241.18it/s]






I: 100%|██████████| 41/41 [00:00<00:00, 68.55it/s]


100%|██████████| 41/41 [00:00<00:00, 244.17it/s]






I: 100%|██████████| 41/41

In [128]:
post_samples = np.array(np.abs(samples)).T
benchmark = []
for i in range(len(dU_error)):
    benchmark.append((np.mean(post_samples[i]),np.std(post_samples[i])))
benchmark = np.array(benchmark)
print('mean:',benchmark[:,0])
print('std:',benchmark[:,1])

mean: [1.34190361e+00 4.22336972e-03 1.01903239e-03 1.25756307e-03
 1.47958723e-03 9.81138906e-04]
std: [0.07016992 0.00295791 0.00069701 0.00087498 0.00078476 0.00062951]
